In [1]:
!pip install soundfile
!pip install sklearn
!pip install tensorflow

You are using pip version 19.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Stored in directory: /Users/fahmisalman/Library/Caches/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn
You are using pip version 19.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.1, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import os
import numpy as np
import pandas as pd
import soundfile as sf
import scipy.signal as signal

## Load training data

In [2]:
train_path = 'dataset/train/'

series = []
length = []
for filename in os.listdir(train_path):
    flac, samplerate = sf.read(train_path+filename)
    series.append(flac)
    length.append(samplerate)
    
label = []
for filename in os.listdir(train_path):
    label.append(filename[:2])
    
data = {'series': series,
       'languange':label}

df = pd.DataFrame(data)
df.head()

,series,languange
0,"[0.00274658203125, 0.004486083984375, 0.010803...",de
1,"[0.07891845703125, 0.09246826171875, 0.1100769...",de
2,"[-0.007354736328125, -0.007415771484375, -0.00...",de
3,"[0.038848876953125, 0.04473876953125, 0.055511...",de
4,"[0.000335693359375, -0.000244140625, 0.0032043...",de


## Load test data

In [38]:
test_path = 'dataset/test/'

series = []
length = []
for filename in os.listdir(test_path):
    flac, samplerate = sf.read(test_path+filename)
    series.append(flac)
    length.append(samplerate)
    
label = []
for filename in os.listdir(test_path):
    label.append(filename[:2])
    
data_test = {'series': series,
       'languange':label}

df_test = pd.DataFrame(data_test)
df_test.head()

,series,languange
0,"[0.0025634765625, 0.002227783203125, 0.0018005...",de
1,"[-0.00830078125, 0.01513671875, -0.02392578125...",de
2,"[6.103515625e-05, 0.0, 0.0, 6.103515625e-05, 0...",de
3,"[0.0067138671875, 0.000152587890625, 0.0022888...",de
4,"[0.03125, 0.0198974609375, 0.015869140625, 0.0...",de


## Data preprocessing

In [39]:
from sklearn.preprocessing import LabelBinarizer
import librosa

def one_hot_encode(x):
    """
    One hot encode a list of sample labels. Return a one-hot encoded vector for each label.
    : x: List of sample Labels
    : return: Numpy array of one-hot encoded labels
    """
    return label_binarizer.transform(x)

x = np.array(df['series'])
x_t = np.array(df_test['series'])
y_train = df['languange']
y_test = df_test['languange']

# Convert Pandas DataFrame into Numpy array
x_train = np.zeros((len(x), 431))
for i in range(len(x_train)):
    a = librosa.feature.zero_crossing_rate(x[i])
    x_train[i] = a
    
x_test = np.zeros((len(x_t), 431))
for i in range(len(x_test)):
    a = librosa.feature.zero_crossing_rate(x_t[i])
    x_test[i] = a
    
# Convert label into one hot encoding
label_binarizer = LabelBinarizer()
label_binarizer.fit(y_train) # need to be global or remembered to use it later
y_train_onehot = one_hot_encode(y_train)

label_binarizer.fit(y_test) # need to be global or remembered to use it later
y_test_onehot = one_hot_encode(y_test)

# Classification

## Naïve Bayes

In [5]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(x_train, y_train)
print(clf.predict(x_test))
print('Train:', clf.score(x_train, y_train))
print('Test :', clf.score(x_test, y_test))

['de' 'de' 'de' 'de' 'de' 'de' 'en' 'de' 'de' 'en' 'en' 'de' 'de' 'de'
 'de' 'de' 'de' 'de' 'en' 'en' 'en' 'de' 'de' 'en' 'de' 'de' 'de' 'en'
 'de' 'de' 'de' 'de' 'de' 'de' 'de' 'de' 'en' 'en' 'en' 'de' 'en' 'en'
 'en' 'en' 'en' 'de' 'en' 'en' 'en' 'en' 'en' 'en' 'en' 'en' 'de' 'en'
 'de' 'de' 'es' 'de' 'de' 'de' 'de' 'de' 'en' 'de' 'en' 'de' 'de' 'de'
 'de' 'de' 'es' 'es' 'es' 'es' 'es' 'es' 'es' 'es' 'es' 'es' 'es' 'es'
 'es' 'es' 'es' 'es' 'es' 'es' 'es' 'es' 'es' 'es' 'de' 'es' 'es' 'de'
 'es' 'es' 'es' 'es' 'es' 'de' 'es' 'de' 'de' 'es']
Train: 0.8148148148148148
Test : 0.7222222222222222


## Neural Network

In [23]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(10), random_state=1)

clf.fit(x_train, y_train_onehot)
print('Train:', clf.score(x_train, y_train_onehot))
print('Test :', clf.score(x_test, y_test_onehot))

Train: 1.0
Test : 0.6481481481481481


In [37]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=5)
clf.fit(x_train, y_train)
print('Train:', clf.score(x_train, y_train))
print('Test :', clf.score(x_test, y_test))

ValueError: Found array with dim 3. Estimator expected <= 2.

In [25]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(x_train, y_train)
print('Train:', clf.score(x_train, y_train))
print('Test :', clf.score(x_test, y_test))

/Users/fahmisalman/.conda/envs/Anaconda3.6/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Train: 0.6782407407407407
Test : 0.6111111111111112


In [36]:
from tensorflow.keras import Sequential, models
from tensorflow.keras.layers import LSTM, Dense, Flatten, GRU
from tensorflow.keras.metrics import binary_accuracy

def train(x, y, epoch=20, hidden=256):
    model = Sequential()
    model.add(GRU(units=hidden, input_shape=(x.shape[1], 1)))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    model.fit(x, y, epochs=epoch, batch_size=64, verbose=1)
    return model


def evaluate(x, y, model):
    mse = model.evaluate(x, y, verbose=0)
    return mse


def save_model(model, s):
    model_json = model.to_json()
    with open("model/%s.json" % s, "w") as json_file:
        json_file.write(model_json)
    model.save_weights("model/%s.h5" % s)


def load_model(s):
    model_json = open('model/%s.json' % s, 'r').read()
    model = models.model_from_json(model_json)
    model.load_weights("model/%s.h5" % s)
    return model


def predict(x, model):
    return model.predict(x)


x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
model = train(x_train, y_train_onehot, epoch=1)
print(evaluate(x_test, y_test_onehot, model))
save_model(model, 'model')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_5 (GRU)                  (None, 256)               198144    
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 771       
Total params: 198,915
Trainable params: 198,915
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/1
432/432 [==============================] - 15s 35ms/step - loss: 0.2208 - acc: 0.3866
[0.21808820576579482, 0.37037037037037035]
